## Quickstart

*Quibbler* allows easily creating highly interactive and efficient data analysis applications. Importantly, such powerful applications are programmed in *Quibbler* using completely standard *Python*, *NumPy* and *Matplotlib* programming syntax, so there is very little to learn to get started.

This page provides a "minimal-demo" to quickly get you up and running. 

For additional demos, consider also the [[Examples]]. 

For a more methodological tour, see [[Introduction]] and the complete [[User Guide]].

Before starting, please first [[install|Installation]] `pyquibbler`.

### Import

`pyquibbler` is conventionally imported as `qb`. In addition, it is convenient to specifically import some often-used functions such as `iquib` (which will be explained below). Following import, we execute `qb.override_all()` which configures *NumPy* and *Matplotlib* functions to work with *Quibbler*. A typical import therefore looks as follows:

In [1]:
# Quibbler import:
import pyquibbler as qb
from pyquibbler import iquib
qb.override_all()

# Other imports:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

### Example of a minimal app

As a quick simple example to demonstrate *Quibbler*, consider the following code for creating a figure with a draggable marker and corresponding text at defined x-y coordinates:

In [2]:
xy = iquib(np.array([250, 250]))
x = xy[0]
y = xy[1]
plt.axis('square')
plt.axis([0, 500, 0, 500])
plt.plot(x, y, marker='o', color='Orange', picker=True)
plt.text(x, y + 20, np.array2string(xy), color='Orange');

[[/images/Quickstart_assign_xy_and_drag.gif]]

As we can see, except for the use of the function `iquib` (which will be explained below), the rest is a completely standard *Python* code for plotting a marker at position x=50, y=50 and a text label with a string representation of these coordinates. Indeed, running this code plots the marker and the text as classically expected. Yet, unlike in classical programming, in *Quibbler* the data items and graphics are all bi-directionally linked. First, assigning new values to an upstream variable, say assigning `xy[0] = 300` immediately refreshes all downstream graphics. Second, the plotted marker is in fact draggable and, as we drag it, we see that it leads to changes in the upstream variable `xy` and in any dependent graphics.

In *Quibbler*, thereby, we can easily get interactive functionality while using completely standard programming syntax and without the need for the tedious programming of event-specific callback functions for creating interactive behaviors.

### How does it work?

Below, we briefly explain the above example, while providing a more general view of *Quibbler* functionality.

#### The quib object
*Quibbler* functionality is based on the *quib* object. The quib is an object that represents an output *value* as well as the *function* and *arguments* used to calculate this value. There are two major types of quibs: input-quibs (i-quibs) which take a regular *Python* object as their argument and present it as their value, and function-quibs (f-quibs) that calculate their output value by applying a given function to a given list of arguments, which could include other quibs and any other *Python* objects. 

#### Input-quibs
Input-quibs are created using the function `iquib`, which transforms any regular *Python* object into a quib. In our case `xy = iquib(np.array([50, 50]))` creates an i-quib `xy` whose value is the *NumPy* array `[50, 50]`. 

#### Function-quibs
Function-quibs are created naturally whenever we use quibs as part of standard expressions or functions. Indeed, *Quibbler* modifies standard functions and operators such that they can work directly with quibs. Such Quibbler-supported functions, also called _quiby functions_, include not only many standard *Python*, *NumPy* and *Matplotlib* functions (see [[Quiby functions]]), but also operators (such as `+`, `-`, `<`, `>`, `**`, `@`), and any array indexing syntax. We can therefore easily define a chained network of function quibs using standard programming syntax. 

In our case, the commands `x = xy[0]` and `y = xy[1]` create the f-quibs `x` and `y` whose function is to reference `xy` at positions 0 and 1, respectively. Next, the command `plt.plot(x, y, ...)` defines an f-quib whose function is to perform `plt.plot` on the values of `x` and `y`. Similarly, `y + 4` is a function quib that adds 4 to the value of `y`, `np.array2string(xy)` is a function-quib that performs the `array2string` on the value of `xy` and, finally, `plt.text(...)` is a function quib that calls `plt.text` with the values of its quib arguments. 

#### Upstream changes automatically propagate to affect the value of downstream quibs
All of these quibs are created *declaratively*: they are functional objects whose value changes upon upstream changes. We can probe the value of any quib using the `get_value` method. When we assign `xy[1] = 70`, we change the value of `xy` which in turn changes the value of `y`, which in turn changes the plot and the text.

#### Interaction with the graphics is inverse-propagated to a change upstream quib values
The relation above can also go backwards. When we indicate `picker=True` in a plot command, *Quibbler* allows draggin the plotted graphics and translates such user interactions with the graphics into assignments to the corresponding quib arguments of the plt.plot function and from there further upstream to input quibs. In our case dragging the marker is translated into assignments to the `x` and `y` quibs. Since `x` and `y` are function quibs, the assignment is further inverse-propagated upstream to the i-quib `xy` where it is actualized (see [[Inverse assignments]]). The change in `xy` then percolates downstream to affect the text position its string label.

### Extending our simple example 

The above principles can be used to build create interactive data analysis pipelines. As a very simple example, we will build a small app for chossing and extracting a square area in an image. 

Let's first plot a square extending length `d` from our defined x-y coordinates:

In [3]:
d = iquib(np.array([120]))
plt.plot(d * np.array([-1, 1, 1, -1, -1]) + x, d * np.array([-1, -1, 1, 1, -1]) + y, 
         color='Orange', picker=True);

[[/images/Quickstart_assign_d_and_drag.gif]]

As we can see, setting `picker=True`, this plot too becomes interactive. Dragging any of the corners of the square is inverted into an assignment to the iquib `d` which then refreshes all other corners of the plotted square (to understand how *Quibbler* chooses to invert these drags into changes in `d` rather than in `x` and `y`, see [[Inverse assignments]], or the example [[quibdemo_drag_whole_object_vs_individual_points]].

### Connecting quibs with widgets

Quibs can also readily connect with *Matplotlib* widgets. Again, unlike in standard programing, using quibs in widgets allows interactive setting of parameters without worrying about defining a callback function for each widget.

As an example, let's add a Slider controlling the size of our square box. 

As the box is extending from -d[0] to d[0], its size is 

In [4]:
box_size = 2*d[0] + 1

To connect widgets with quibs, we simply use standard *Matplotlib* widget syntax with the quib as the initial value of the widget. In our case, we will set a Slider where the initial value of the slider is set to our box_size (`initval=box_size`):

In [5]:
from matplotlib.widgets import Slider
main_axs = plt.gca()
slider_axs = plt.axes([0.2, 0.05, 0.5, 0.04])
Slider(ax=slider_axs, label='box_size', valmin=1, valmax=500, valinit=box_size);

[[/images/Quickstart_widget_box_size.gif]]

**How does it work.** As the slider value is defined as `box_size` its position changes when `box_size` changes (as we drag the square corners). Conversely, when we drag the slider, these interactions are inverted into assignments to `box_size` and, since `box_size` is a function quib, the change further propagate to affect `d`. Note that as `d` is defined as an array of integers, `box_size`, which is defined as `2*d[0]+1` is always an odd number (even as we drag the slider).

### Interactively cutting an image by the defined square

We will now load and plot an image. Defining the file name as a quib, all downstream variables that depend on the file name are then defined as function quibs:

In [11]:
filename = iquib('bacteria_in_droplets.tif')
img = plt.imread(filename)
main_axs.imshow(img);

Now, we can cut the image based on our defined box:

In [ ]:
img_cut = img[y-d[0]:y+d[0]+1, x-d[0]:x+d[0]+1, :]
plt.figure()
plt.imshow(img_cut, origin='lower');

[[/images/Quickstart_interactive_image_cut.gif]]